In [1]:
import pandas as pd 
import numpy as np 
import sklearn.preprocessing
import imblearn

In [104]:
dfTrain = pd.read_csv("train.csv")
dfTest = pd.read_csv("test.csv")
testLabels = pd.read_csv("submission.csv")

In [83]:
dfTrain
X_train, y_train = dfTrain.loc[:, dfTrain.columns[:-1]], dfTrain["subscribed"]
X_test = dfTest.loc[:, dfTest.columns]

In [84]:
X_train.drop("id", axis=1, inplace=True)
X_test.drop("id", axis=1, inplace=True)

In [85]:
def sin_transformer(period):
	return sklearn.preprocessing.FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
	return sklearn.preprocessing.FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

def encodingPipeline(X_train): # Should just transform for X_test, for now it also fits it
    X_category_encoded = pd.get_dummies(X_train, columns=['marital', 'loan', 'housing', 'poutcome'], drop_first=True)
    jobEncoder = sklearn.preprocessing.OrdinalEncoder(categories=[["unknown", "unemployed", "student", "housemaid", "services", "admin.", "technician", "self-employed", "retired", "entrepreneur", "management", "blue-collar"]])
    X_category_encoded["job"] = jobEncoder.fit_transform(np.array(X_category_encoded["job"]).reshape(-1, 1))
    eduEncoder = sklearn.preprocessing.OrdinalEncoder(categories=[["unknown", "illiterate", "basic.4y", "basic.6y", "basic.9y", "high.school", "professional.course", "university.degree"]])
    X_category_encoded["education"] = eduEncoder.fit_transform(np.array(X_category_encoded["education"]).reshape(-1, 1))
    binaryEncoder = sklearn.preprocessing.OrdinalEncoder()
    X_category_encoded["contact"] = binaryEncoder.fit_transform(np.array(X_category_encoded["contact"]).reshape(-1, 1))
    monthEncoder = sklearn.preprocessing.OrdinalEncoder(categories=[["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]])
    X_category_encoded["month"] = monthEncoder.fit_transform(np.array(X_category_encoded["month"]).reshape(-1, 1))
    dayEncoder = sklearn.preprocessing.OrdinalEncoder(categories=[["mon", "tue", "wed", "thu", "fri"]])
    X_category_encoded["day_of_week"] = dayEncoder.fit_transform(np.array(X_category_encoded["day_of_week"]).reshape(-1, 1))
    X_category_encoded["default"] = binaryEncoder.fit_transform(np.array(X_category_encoded["default"]).reshape(-1, 1))

    X_category_encoded["month_sin"] = sin_transformer(12).fit_transform(X_category_encoded)["month"]
    X_category_encoded["month_cos"] = cos_transformer(12).fit_transform(X_category_encoded)["month"]

    X_category_encoded["day_sin"] = sin_transformer(5).fit_transform(X_category_encoded)["day_of_week"]
    X_category_encoded["day_cos"] = cos_transformer(5).fit_transform(X_category_encoded)["day_of_week"]
    X_category_encoded.drop(labels=["month", "day_of_week"], inplace=True, axis=1)
    return X_category_encoded
    

    

In [86]:
categoryCols = X_train.select_dtypes(exclude=np.number).columns
binaryEncoder = sklearn.preprocessing.OrdinalEncoder()
y_train = binaryEncoder.fit_transform(np.array(y_train).reshape(-1, 1))

In [87]:
y_train.ravel()

array([0., 0., 0., ..., 0., 0., 0.])

In [88]:

X_train_numeric = X_train.select_dtypes(include=np.number)
X_train_encoded = encodingPipeline(X_train[categoryCols])
X_train = pd.concat([X_train_numeric, X_train_encoded], axis=1)

In [20]:
import sklearn.model_selection

X = X_train
y = y_train
sss = sklearn.model_selection.StratifiedShuffleSplit(10, test_size=0.3, random_state=13)
X = X_train
y = pd.DataFrame(y_train)
for train_index, test_index in sss.split(X, y):
    X_train, X_test_fake = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test_fake = y.iloc[train_index], y.iloc[test_index]

In [47]:
X_test_fake

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,job,...,loan_unknown,loan_yes,housing_unknown,housing_yes,poutcome_nonexistent,poutcome_success,month_sin,month_cos,day_sin,day_cos
15390,32,2,999,0,1.4,94.465,-41.8,4.866,5228.1,6.0,...,False,False,False,False,True,False,0.500000,-8.660254e-01,-0.587785,-0.809017
27502,43,1,999,0,1.4,94.465,-41.8,4.959,5228.1,11.0,...,False,True,False,False,True,False,0.500000,-8.660254e-01,-0.951057,0.309017
25007,30,2,999,0,-1.8,92.893,-46.2,1.266,5099.1,5.0,...,False,True,False,False,True,False,0.866025,-5.000000e-01,-0.587785,-0.809017
29975,48,2,5,2,-3.4,92.431,-26.9,0.742,5017.5,5.0,...,False,False,False,True,False,True,-1.000000,-1.836970e-16,0.951057,0.309017
11854,36,5,999,0,-1.8,92.893,-46.2,1.344,5099.1,11.0,...,False,False,False,True,True,False,0.866025,-5.000000e-01,0.951057,0.309017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4490,33,4,999,0,-1.8,92.893,-46.2,1.281,5099.1,4.0,...,False,False,False,False,True,False,0.866025,-5.000000e-01,0.587785,-0.809017
5077,36,2,999,0,-0.1,93.200,-42.0,4.191,5195.8,6.0,...,False,False,False,False,True,False,-0.866025,5.000000e-01,0.000000,1.000000
8371,49,5,999,0,1.4,93.444,-36.1,4.965,5228.1,5.0,...,False,False,False,False,True,False,-0.500000,-8.660254e-01,0.951057,0.309017
14975,52,1,999,0,-0.1,93.200,-42.0,4.153,5195.8,4.0,...,False,False,False,True,True,False,-0.866025,5.000000e-01,0.951057,0.309017


In [22]:
y_train.value_counts()

0.0    18617
1.0     2383
Name: count, dtype: int64

In [23]:
y_test_fake.value_counts()

0.0    7979
1.0    1021
Name: count, dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTE

su = SMOTE(random_state=42, k_neighbors=4)
X_train, y_train = su.fit_resample(X_train, y_train)
X_train.shape, y_train.shape

((37234, 26), (37234, 1))

In [25]:
pd.DataFrame(y_train).value_counts()

0.0    18617
1.0    18617
Name: count, dtype: int64

In [26]:
import sklearn.ensemble

In [36]:
# Create adaboost classifer object
abc = sklearn.ensemble.AdaBoostClassifier(n_estimators=50,
                         learning_rate=1, random_state=13)
# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test_fake)

e:\Anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [37]:
import sklearn.metrics


sklearn.metrics.f1_score(y_test_fake, y_pred)

0.4246383574428372

In [31]:
import xgboost as xgb

xgb_clf = xgb.XGBClassifier(
    n_estimators=100,  # Number of trees in the ensemble
    learning_rate=0.1,  # Step size for updating weights
    max_depth=3,       # Maximum depth of each tree
    random_state=42    # Ensures reproducibility
)

In [32]:
# Train Adaboost Classifer
model = xgb_clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test_fake)

In [33]:
sklearn.metrics.f1_score(y_test_fake, y_pred)

0.4443324937027708

In [45]:
import optuna
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
    }

    # Create and train the model
    xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42, **params)
    scores = cross_val_score(xgb_clf, X_train, y_train, cv=3, scoring="f1")
    
    # Return the mean accuracy
    return scores.mean()

# Create a study object and optimize
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Get the best hyperparameters
print("Best hyperparameters:", study.best_params)

# Train the final model with the best hyperparameters
best_params = study.best_params
final_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42, **best_params)
final_model.fit(X_train, y_train)


[I 2024-12-04 16:24:21,004] A new study created in memory with name: no-name-7aec0dd9-f86d-449d-a90a-b7c512ae5502
e:\Anaconda\lib\site-packages\xgboost\core.py:158: UserWarning: [16:24:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
e:\Anaconda\lib\site-packages\xgboost\core.py:158: UserWarning: [16:24:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
e:\Anaconda\lib\site-packages\xgboost\core.py:158: UserWarning: [16:24:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  w

Best hyperparameters: {'n_estimators': 293, 'max_depth': 8, 'learning_rate': 0.03336625934976126, 'subsample': 0.8037892683422048, 'colsample_bytree': 0.9672254241488442, 'gamma': 0.00986818410724289, 'reg_alpha': 0.9794671362002928, 'reg_lambda': 0.9006824748069236}


e:\Anaconda\lib\site-packages\xgboost\core.py:158: UserWarning: [16:31:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9672254241488442, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', feature_types=None,
              gamma=0.00986818410724289, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03336625934976126,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=293, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
# Evaluate the model on the test set
y_pred = final_model.predict(X_test_fake)
accuracy = sklearn.metrics.f1_score(y_test_fake, y_pred)
print("Test set score:", accuracy)

Test set score: 0.414605418138987


In [52]:
import json
y_test = ""
with open("message.txt", "r") as f:
    y_test = json.load(f)

print(y_test)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 

In [90]:
X_test_numeric = X_test.select_dtypes(include=np.number)
X_test_encoded = encodingPipeline(X_test[categoryCols])
X_test = pd.concat([X_test_numeric, X_test_encoded], axis=1)

In [56]:
y_pred = final_model.predict(X_test)
print(sklearn.metrics.f1_score(y_test, y_pred))

0.4161290322580645


In [59]:
xgb_clf = xgb.XGBClassifier(
    n_estimators=100,  # Number of trees in the ensemble
    learning_rate=0.1,  # Step size for updating weights
    max_depth=3,       # Maximum depth of each tree
    random_state=42    # Ensures reproducibility
)
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [60]:
y_pred = xgb_clf.predict(X_test)
print(sklearn.metrics.f1_score(y_test, y_pred))

0.4616100511865984


In [109]:
def scalingPipelineFilter(dfTrain, dfTest, filter = True):
    numColumns = dfTrain.select_dtypes(include = np.number).columns
    if filter:
        for col in numColumns:
            dfTrain.loc[:,'z_score' + col] = (dfTrain[col] - dfTrain[col].mean())/dfTrain[col].std()
            dfTrain = dfTrain.loc[(dfTrain['z_score' + col] <= 2) & (dfTrain['z_score' + col] >= -2),:]
            dfTrain.drop('z_score' + col, axis=1, inplace=True)
    import sklearn.preprocessing
    cols = numColumns[1:] # skip age
    standardScaler = sklearn.preprocessing.StandardScaler()
    robustScaler = sklearn.preprocessing.RobustScaler()
    #X_train_scaled = X_train_numeric
    dfTrain["age"] = standardScaler.fit_transform(np.array(dfTrain["age"]).reshape(-1,1))
    dfTest["age"] = standardScaler.transform(np.array(dfTest["age"]).reshape(-1,1))
    dfTrain[cols] = robustScaler.fit_transform(dfTrain[cols])
    dfTest[cols] = robustScaler.transform(dfTest[cols])
    return dfTrain, dfTest

In [117]:
import sklearn.preprocessing


def encodingPipeline2(dfTrain, dfTest):
    dfTrain['subscribed'] = binaryEncoder.fit_transform(np.array(dfTrain['subscribed']).reshape(-1, 1))
    categorical_features = dfTrain.select_dtypes(include=['object']).columns
    ordered_categories = {}
    for feature in categorical_features:
        class_ratios = (dfTrain.groupby(feature)['subscribed'].mean().sort_values())
        ordered_categories[feature] = list(class_ratios.index)
    encoder = sklearn.preprocessing.OrdinalEncoder(categories=[ordered_categories[feature] for feature in categorical_features])
    dfTrain[categorical_features] = encoder.fit_transform(dfTrain[categorical_features])
    dfTest[categorical_features] = encoder.transform(dfTest[categorical_features])
    return dfTrain, dfTest

In [126]:
dfTrain = pd.read_csv("train.csv")
dfTest = pd.read_csv("test.csv")
testLabels = pd.read_csv("submission.csv")

In [127]:
X_train, X_test = encodingPipeline2(dfTrain, dfTest)
X_train.drop("id", axis = 1, inplace = True)
X_test.drop("id", axis = 1, inplace = True)
X_train, X_test = scalingPipelineFilter(X_train, X_test)

C:\Users\Michal\AppData\Local\Temp\ipykernel_8932\3432697105.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTrain.drop('z_score' + col, axis=1, inplace=True)
C:\Users\Michal\AppData\Local\Temp\ipykernel_8932\3432697105.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTrain.loc[:,'z_score' + col] = (dfTrain[col] - dfTrain[col].mean())/dfTrain[col].std()


ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by StandardScaler.

In [120]:
y_train = X_train["subscribed"].ravel()
X_train.drop("subscribed", axis = 1, inplace=True)

In [122]:
X_train

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,0.714286,unemployed,married,basic.9y,no,yes,yes,telephone,jun,thu,-0.5,0.0,0.0,nonexistent,0.0,0.994545,0.000000,-0.028302,0.0
2,-1.000000,admin.,single,basic.9y,no,no,no,cellular,jul,mon,-0.5,0.0,0.0,nonexistent,0.0,0.000000,-0.160714,0.018868,0.0
3,0.285714,services,married,basic.6y,unknown,no,no,cellular,jul,mon,-0.5,0.0,0.0,nonexistent,0.0,0.000000,-0.160714,0.037736,0.0
5,-0.571429,admin.,married,university.degree,unknown,no,no,cellular,aug,mon,-0.5,0.0,0.0,nonexistent,0.0,-0.861818,1.017857,0.066038,0.0
6,1.142857,entrepreneur,married,professional.course,unknown,yes,no,cellular,jul,fri,0.0,0.0,0.0,nonexistent,0.0,0.000000,-0.160714,0.047170,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29989,-0.285714,admin.,single,university.degree,no,no,no,cellular,aug,tue,0.0,0.0,0.0,nonexistent,0.0,-0.861818,1.017857,0.094340,0.0
29994,0.714286,admin.,married,university.degree,unknown,yes,no,telephone,jul,mon,-0.5,0.0,0.0,nonexistent,0.0,0.000000,-0.160714,0.018868,0.0
29995,-0.214286,admin.,married,university.degree,no,yes,yes,telephone,may,tue,0.0,0.0,0.0,nonexistent,-1.0,0.138182,0.964286,-0.952830,-1.0
29996,0.500000,technician,divorced,university.degree,unknown,yes,no,telephone,jul,fri,1.0,0.0,0.0,nonexistent,0.0,0.000000,-0.160714,0.037736,0.0


In [101]:
from imblearn.combine import SMOTEENN

su = SMOTEENN(random_state=42, sampling_strategy=0.98)
X_train, y_train = su.fit_resample(X_train, y_train)
X_train.shape, y_train.shape

((42034, 19), (42034,))

In [102]:
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)
print(sklearn.metrics.f1_score(y_test, y_pred))

0.49644859813084113


In [77]:
predictions = y_pred
predictions = [int(i) for i in predictions]
pd.DataFrame({"subscribed_predicted": predictions}).to_csv("submission2.csv", index_label="id")

In [78]:
len(y_pred)

10000

In [103]:
import sklearn.ensemble


rf = sklearn.ensemble.RandomForestClassifier(n_estimators= 300, max_depth= 29, min_samples_split= 12, min_samples_leaf = 5, max_features= 'sqrt')
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(sklearn.metrics.f1_score(y_test, y_pred))

0.4965716347888849


In [ ]:
import sklearn.tree


dtc = sklearn.tree.DecisionTreeClassifier()
svc = 